In [1]:
using JLD2
using Statistics
using Revise
using Plots 
using Laplacians

In [2]:
function nanErr!(d)
    for (key, value) in d
        if occursin("err", key)
            for i in 1:length(value)
                if isnan(value[i])
                    d[key][i] = Inf
                end
            end
        end
    end
end

nanErr! (generic function with 1 method)

In [4]:
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n10000.h1.0.jld2"
dic1 = copy(dic)
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n100000.h2.0.jld2"
dic2 = copy(dic)
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n1000000.h4.0.jld2"
dic3 = copy(dic)
@load "wted_bndry_chimera_nopetsc.jl.split02merge02.n10000000.h8.0.jld2"
dic4 = copy(dic)

Dict{Any,Any} with 44 entries:
  "hypre_solve" => [261.05, 99.43, Inf, 72.28, 188.39, 85.58, 158.43, 115.97]
  "icc2_tot"    => [500.419, 104.342, 47.5152, 1076.5, 227.689, 211.438, 262.14…
  "lamg_its"    => [Inf, Inf, Inf, 9.0, 9.0, 8.0, 9.0, Inf]
  "ac_tot"      => [215.737, 232.003, 117.224, 196.134, 440.152, 306.406, 414.5…
  "icc2_solve"  => [485.394, 84.8441, 29.9954, 1059.8, 205.836, 189.837, 239.75…
  "hypre_tot"   => [484.44, 349.89, Inf, 186.12, 528.59, 284.14, 406.41, 346.23]
  "ac-s2m2_its" => [33.0, 16.0, 21.0, 26.0, 27.0, 18.0, 34.0, 21.0]
  "icc_solve"   => [449.715, 77.3485, 23.1996, 919.366, 218.206, 147.946, 194.5…
  "icc2_its"    => [435.0, 58.0, 24.0, 989.0, 133.0, 114.0, 138.0, 131.0]
  "cmg2_solve"  => [148.448, 219.033, 128.671, 182.472, 250.316, 298.207, 227.4…
  "icc_build"   => [19.8824, 29.7492, 21.6895, 22.2964, 35.1043, 35.4509, 34.02…
  "cmg_err"     => [6.89038e-9, 7.30111e-9, 8.72864e-9, 6.7019e-9, 8.24937e-9, …
  "lamg_err"    => [Inf, Inf, Inf, 0.0003

In [5]:
nanErr!(dic1);
nanErr!(dic2);
nanErr!(dic3);
nanErr!(dic4);

In [6]:
using Printf

function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)

ffmicro (generic function with 1 method)

In [7]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printMedian(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = median(d[string(solver, "_tot")] ./ d["ne"] )
        err = median(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function print75(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = quantile(d[string(solver, "_tot")] ./ d["ne"], 0.75)
        err = quantile(d[string(solver, "_err")], 0.75)
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function printMax(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = maximum(d[string(solver, "_tot")] ./ d["ne"])
        err = maximum(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

printMax (generic function with 1 method)

In [8]:
function printSolverCell(d, solver) 
    return string(printMedian(d, solver), "&", print75(d, solver), "&", printMax(d, solver))
end

function printOneLine(d, solvers)
    allCells = [printSolverCell(d, solver) for solver in solvers]
    println(string(ffK(d["nv"][1]), "&", length(d["testName"]), "&", join(allCells, "&"), "\\\\"))
end

function printAllLines(dics, solvers)
    for d in dics
        printOneLine(d, solvers)
    end
end

printAllLines (generic function with 1 method)

In [9]:
dics = [dic1, dic2, dic3, dic4];

In [10]:
solvers=["ac", "ac-s2m2", "cmg2"]
printAllLines(dics, solvers)

9.54&103&0.375&0.385&0.431&0.537&0.592&0.714&8.58&10.5&16.8\\
97.9&105&0.606&0.701&1.43&0.909&1.11&1.82&1.88&2.46&3.83\tnote{*}\\
990&23&1.4&1.59&2.16&1.66&2.14&2.42&1.84&2.57&Inf\\
9950&8&3.07&3.99&4.34&3.34&4.18&4.36&2.91&3.56&4.19\\


In [11]:
solvers = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dics, solvers)

9.54&103&0.873&1.26&2.4&N/A&N/A&N/A&4.68&5.73&9.3\\
97.9&105&0.946&1.66&4.73&N/A&N/A&N/A&1.14&1.72&10.7\tnote{*}\\
990&23&2.61&4.29&11.4\tnote{*}&N/A&N/A&N/A&1.36&3.37&Inf\\
9950&8&4.39&6.4&Inf&N/A&N/A&N/A&2.31&4.47&16.6\tnote{*}\\
